In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
df=pd.read_csv('u.data', sep='\t',names=['user_id','item_id','ratings','timestamp'])
df.head()

,user_id,item_id,ratings,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [10]:
movies=pd.read_csv('Movie_Id_Titles')
movies.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [15]:
movie_ratings=pd.merge(df,movies,on='item_id')
movie_ratings.head()

,user_id,item_id,ratings,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,290,50,5,880473582,Star Wars (1977)
2,79,50,4,891271545,Star Wars (1977)
3,2,50,5,888552084,Star Wars (1977)
4,8,50,5,879362124,Star Wars (1977)


In [81]:
rating=pd.DataFrame(movie_ratings.groupby('title')['ratings'].mean())
rating['number_of_ratings']=movie_ratings.groupby('title')['ratings'].count()
rating

,ratings,number_of_ratings
title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41
...,...,...
Young Guns II (1990),2.772727,44
"Young Poisoner's Handbook, The (1995)",3.341463,41
Zeus and Roxanne (1997),2.166667,6


In [36]:
movie_list=movie_ratings.pivot_table(index='user_id',columns='title',values='ratings')

In [37]:
movie_list.keys()


Index([''Til There Was You (1997)', '1-900 (1994)', '101 Dalmatians (1996)',
       '12 Angry Men (1957)', '187 (1997)', '2 Days in the Valley (1996)',
       '20,000 Leagues Under the Sea (1954)', '2001: A Space Odyssey (1968)',
       '3 Ninjas: High Noon At Mega Mountain (1998)', '39 Steps, The (1935)',
       ...
       'Yankee Zulu (1994)', 'Year of the Horse (1997)', 'You So Crazy (1994)',
       'Young Frankenstein (1974)', 'Young Guns (1988)',
       'Young Guns II (1990)', 'Young Poisoner's Handbook, The (1995)',
       'Zeus and Roxanne (1997)', 'unknown',
       'Á köldum klaka (Cold Fever) (1994)'],
      dtype='object', name='title', length=1664)

In [73]:
young_guns=movie_list['Young Guns (1988)']
young_guns=pd.DataFrame(young_guns)
#young_guns.dropna('Young Guns (1988)')
young_guns

,Young Guns (1988)
user_id,
0,NaN
1,3.0
2,NaN
3,NaN
4,NaN
...,...
939,NaN
940,NaN
941,NaN


In [74]:
corr=movie_list.corrwith(movie_list['Young Guns (1988)'])
corr.sort_values(ascending=False).head()

C:\Users\anike\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\anike\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


title
Nothing to Lose (1994)    1.0
8 Seconds (1994)          1.0
Set It Off (1996)         1.0
Denise Calls Up (1995)    1.0
Picture Bride (1995)      1.0
dtype: float64

In [87]:
similar=pd.DataFrame(corr,columns=['Correlation'])
similar

,Correlation
title,
'Til There Was You (1997),NaN
1-900 (1994),NaN
101 Dalmatians (1996),0.119234
12 Angry Men (1957),0.068944
187 (1997),-0.500000
...,...
Young Guns II (1990),0.722460
"Young Poisoner's Handbook, The (1995)",-0.019672
Zeus and Roxanne (1997),NaN


In [88]:
similar.dropna(inplace=True)
similar

,Correlation
title,
101 Dalmatians (1996),0.119234
12 Angry Men (1957),0.068944
187 (1997),-0.500000
2 Days in the Valley (1996),0.156220
"20,000 Leagues Under the Sea (1954)",-0.010894
...,...
Year of the Horse (1997),1.000000
Young Frankenstein (1974),0.510284
Young Guns (1988),1.000000


In [89]:
similar = similar.join(rating['number_of_ratings'])
similar

,Correlation,number_of_ratings
title,,
101 Dalmatians (1996),0.119234,109
12 Angry Men (1957),0.068944,125
187 (1997),-0.500000,41
2 Days in the Valley (1996),0.156220,93
"20,000 Leagues Under the Sea (1954)",-0.010894,72
...,...,...
Year of the Horse (1997),1.000000,7
Young Frankenstein (1974),0.510284,200
Young Guns (1988),1.000000,101


In [96]:
similar.dropna(inplace=True)

In [100]:
similar_movies=similar[similar['number_of_ratings']>100].sort_values('Correlation',ascending=False)

In [103]:
similar_movies[similar_movies['Correlation']>0.5]

,Correlation,number_of_ratings
title,,
Young Guns (1988),1.000000,101
Good Will Hunting (1997),0.675082,198
"River Wild, The (1994)",0.662424,146
"Frighteners, The (1996)",0.629871,115
Seven Years in Tibet (1997),0.599263,155
"Time to Kill, A (1996)",0.597969,232
"Ghost and the Darkness, The (1996)",0.585139,128
"Nightmare on Elm Street, A (1984)",0.578651,111
"First Wives Club, The (1996)",0.569725,160
